## Setup

First, let's make sure we have the proper version installed. This notebook works in python 3. Import a few common modules.

In [ ]:
pip install torch

In [ ]:
pip install ultralytics

In [ ]:
import os                                # For operating system dependent functionality
import cv2                               # OpenCV library for image processing
import gc                                # Garbage collector for memory management
import torch                             # PyTorch deep learning framework
import numpy as np                       # NumPy for numerical computations
import motmetrics as mm                  # Metrics for multiple object tracking

import matplotlib                        # Matplotlib for data visualization
import matplotlib.pyplot as plt          # Plotting graphs and figures

import tensorflow as tf                  # TensorFlow deep learning framework
import tensorflow_datasets as tfds       # TensorFlow Datasets library
import flow_vis                          # Optical flow visualization
from flow_util import *                  # Flow utilities
import matplotlib.patches as patches     # For drawing shapes on plots

In [1]:
from pkg_resources import parse_version  # Importing version parsing from package resources
from PIL import Image                    # Importing Image module from the Python Imaging Library (PIL)

# Python ≥3.5 is required
import sys                               # Importing system-related functionalities
assert sys.version_info >= (3,5)         # Assertion to check if the Python version is 3.5 or higher

from google.colab import drive           # Importing Google Colab drive functionalities
from ultralytics import YOLO             # Importing YOLO object detection module from Ultralytics

This code defines a set of utility functions related to setting up the environment, saving figures, and generating log directories.

In [ ]:
# to make this notebook's output stable across runs
def reset_seed(seed=42):
    np.random.seed(seed)

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
# Based on post by GM at https://stackoverflow.com/questions/53581278
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  from google.colab import drive
  # mount your google drive into colab
  # this is interactive unfortunately
  # must paste authorization code on prompt
  drive.mount('/content/drive')
  PROJECT_ROOT_DIR = "drive/MyDrive/Colab Notebooks"
else:
  print('Not running on CoLab')
  PROJECT_ROOT_DIR = os.getcwd()

NB_ID = "project"

# create the directory if it does not exist
IMAGE_DIR = os.path.join(PROJECT_ROOT_DIR, "images", NB_ID)
os.makedirs(IMAGE_DIR, exist_ok = True)

def save_fig_nb(fig_id, tight_layout=True):
    path = os.path.join(IMAGE_DIR, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

os.makedirs(os.path.join(PROJECT_ROOT_DIR,"tb_logs",NB_ID), exist_ok = True)

def get_logdir(add=''):
    import time
    log_id = time.strftime("log_%Y_%m_%d-%H_%M_%S")
    return os.path.join(PROJECT_ROOT_DIR,"tb_logs",NB_ID,log_id+add)

Not running on CoLab


A couple utility functions to plot grayscale and RGB images (source Aurelien Geron):

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image.astype(np.uint8),interpolation="nearest")
    plt.axis("off")

## Import tfds and other libraries

In [ ]:
# For some reason we seem to need to load tf onto the GPU first
dummy = tf.constant( 32 )
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2023-11-30 23:33:48.457661: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 23:33:48.487209: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 23:33:48.487656: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 23:33:49.005772: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/luminatech/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Num GPUs Available:  0


2023-11-30 23:33:49.887974: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-30 23:33:49.913627: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Data Loading

The dataset is packaged as a Tensorflow dataset in three sub categories.


In [ ]:
tfds.disable_progress_bar()

print(PROJECT_ROOT_DIR)

data_path = os.path.join(PROJECT_ROOT_DIR, 'tensorflow_data')
os.makedirs(data_path, exist_ok = True)

/home/luminatech/Documents/uOttawa/Computer_Vision/Tabinng and Testing/Testing_and_Training


# YoloV8 dataset

## Images dataset
Exporting images from tfrecords and splitting into train, validation, and test datasets in different folders

In [ ]:
tfds.disable_progress_bar()

print("Using directory: ", data_path)

# Dataset names for each fold
fold_names = ['fixed_random_rotate', 'linear_movement_rotate', 'rotation_rotate']

# Create a dictionary to store the datasets for each fold
fold_datasets = {}

for fold_name in fold_names:
    # Load the dataset for the current fold
    dataset = tfds.load(f'elg7186_projectdata/{fold_name}', data_dir=data_path)
    fold_datasets[fold_name] = dataset

# Create a list to store all images in the fold
images_train = []
images_val = []
images_test = []

# Access images from each fold
for fold_name, dataset in fold_datasets.items():

    # Iterate over all examples in the fold
    vid_num = 0
    for example in dataset['train']:
        # Get the video from the example
        video = example['video']

        # Iterate over all frames in the video and add them to the list
        frame_num = 0
        for frame in video:
            if frame_num <16:
                images_train.append(frame.numpy())
                img = Image.fromarray(frame.numpy())
                img.save(f'./datasets/data/train/images/{fold_name}_video_{vid_num}_frame_{frame_num}.png')
            elif frame_num<18 :
                images_val.append(frame.numpy())
                img = Image.fromarray(frame.numpy())
                img.save(f'./datasets/data/val/images/{fold_name}_video_{vid_num}_frame_{frame_num}.png')
            else:
                images_test.append(frame.numpy())
                img = Image.fromarray(frame.numpy())
                img.save(f'./datasets/data/test/images/{fold_name}_video_{vid_num}_frame_{frame_num}.png')

            frame_num+=1
        vid_num += 1
    print(f"Total videos in {fold_name}: {len(dataset['train'])}")

print(f"Total images in train folder: {len(images_train)}")
print(f"Total images in val folder: {len(images_val)}")
print(f"Total images in test folder: {len(images_test)}")

Using directory:  /home/luminatech/Documents/uOttawa/Computer_Vision/Tabinng and Testing/Testing_and_Training/tensorflow_data


Total videos in fixed_random_rotate: 30
Total videos in linear_movement_rotate: 30
Total videos in rotation_rotate: 30
Total images in train folder: 1440
Total images in val folder: 180
Total images in test folder: 540


## Labels dataset
Exporting labels from tfrecords and splitting into train, validation, and test datasets in different folders

In [ ]:
def find_index(input, id):
    # Function to find the index of 'id' in 'input'
    for i in range(len(input)):
        if input[i] == id:
            return i
            break

gt_mot = []

# Iterate through the 'fold_datasets' dictionary
for fold_name, dataset in fold_datasets.items():
    # Iterate through the 'train' dataset within each 'fold_name'
    vid_num = 0
    for example in dataset['train']:
        # Extract video name and type from example metadata
        video_name, video_type = get_video_names(example['metadata'])

        # Extract scale and offset from example metadata
        f_scale, f_offset = get_scale_offset(example['metadata'])

        # Extract number of frames and resolution from example metadata
        num_frames = int(example['metadata']['num_frames'])
        resolution = [example['metadata']['height'], example['metadata']['width']]

        # Print resolution
        print('resolution:', resolution)

        # Initialize arrays for x_mesh and y_mesh
        x_mesh = np.empty(1)
        y_mesh = np.empty(1)

        trk_id= dict()
        trk_num = 0

        dict_1 = {}

        # Loop through each frame in the video
        for i in range(num_frames):

            # Get the image for the current frame
            image = example['video'][i, :, :, :]

            # Create a subplot for the boundimg boxes
            segmentation = example['segmentations'][i, :, :, :].numpy()

            if i<16:
                f= open(f"./datasets/data/train/labels/{fold_name}_video_{vid_num}_frame_{i}.txt","w+")
            elif i<18:
                g= open(f"./datasets/data/val/labels/{fold_name}_video_{vid_num}_frame_{i}.txt","w+")
            else:
                h= open(f"./datasets/data/test/labels/{fold_name}_video_{vid_num}_frame_{i}.txt","w+")
            # Loop through the instances in the frame

            dict_2 = {}
            min_dist = 0
            for id in range(example['instances']['bbox_frames'].shape[0]):
                if i in example['instances']['bbox_frames'][id]:
                    t = find_index(example['instances']['bbox_frames'][id], i)
                    item = example['instances']['bboxes'][id, t, :]
                    cls_id = example['instances']['category'][id]
                    cls_name = example['instances']['asset_id'][id]
                    y_min = int(item[0] * float(resolution[0]))/example['metadata']['height']
                    x_min = int(item[1] * float(resolution[1]))/example['metadata']['width']
                    y_max = int(item[2] * float(resolution[0]))/example['metadata']['height']
                    x_max = int(item[3] * float(resolution[1]))/example['metadata']['width']

                    if id < 1:
                        if trk_num not in dict_1:
                            dict_1[trk_num] = (x_min,y_min,(x_max-x_min),(y_max-y_min))
                    else:
                        for key, value in dict_1:
                            if abs(value - (x_min,y_min)) < min_dist:
                                min_dist = abs(value - (x_min,y_min))
                                min_key = key
                        dict_2[min_key] = (x_min,y_min)
                        dict_1 = {}
                        for key, value in dict_2:
                            dict_1[key] = value

                    for key, value in dict_1:
                        gt_mot.append([i, key, x_min, y_min, (x_max-x_min), (y_max-y_min)])


                    # Create a Rectangle patch for bounding box
                    rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=1, edgecolor='r', facecolor='none')
                    if i<16:
                        f.write(f"{id} {(x_max-x_min)/2 + x_min} {(y_max-y_min)/2 + y_min} {(x_max-x_min)} {(y_max-y_min)}\n")
                    elif i<18:
                        g.write(f"{id} {(x_max-x_min)/2 + x_min} {(y_max-y_min)/2 + y_min} {(x_max-x_min)} {(y_max-y_min)}\n")
                    else:
                        h.write(f"{id} {(x_max-x_min)/2 + x_min} {(y_max-y_min)/2 + y_min} {(x_max-x_min)} {(y_max-y_min)}\n")
                    trk_num += 1
                else:
                    print("")  # empty line
        vid_num += 1


Video:  0003
Video type:  fixed_random_rotate_007
forward_flow range -16.59033203125 to 16.885251998901367
resolution: [<tf.Tensor: shape=(), dtype=int32, numpy=256>, <tf.Tensor: shape=(), dtype=int32, numpy=256>]


Video:  0006
Video type:  fixed_random_rotate_003
forward_flow range -15.715254783630371 to 53.34239959716797
resolution: [<tf.Tensor: shape=(), dtype=int32, numpy=256>, <tf.Tensor: shape=(), dtype=int32, numpy=256>]










Video:  0001
Video type:  fixed_random_rotate_003
forward_flow range -13.709030151367188 to 19.553253173828125
resolution: [<tf.Tensor: shape=(), dtype=int32, numpy=256>, <tf.Tensor: shape=(), dtype=int32, numpy=256>]


KeyboardInterrupt: 

# Model Training and Testing

### Initializing YOLOv8 Model with Ultralytics Library

In [ ]:
model = YOLO("yolov8n.pt")
model = YOLO("yolov8x.pt")

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/home/luminatech/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'.


### Training YOLOv8 Model on the Kubric Dataset

In [ ]:
model.train(data = "data.yaml", epochs = 1, batch = 8)

### Image Loading Function with OpenCV: Load Images from Folder

In [ ]:
def load_images_from_folder(folder):
    """
    Load images from a specified folder.

    Parameters:
    - folder (str): Path to the folder containing images.

    Returns:
    - images (list): List of images loaded from the folder.
    """
    images = []

    # Iterate through each file in the folder
    for filename in sorted(os.listdir(folder)):
        # Read the image using OpenCV
        img = cv2.imread(os.path.join(folder, filename))

        # Check if the image is not None (i.e., if it was successfully loaded)
        if img is not None:
            # Append the loaded image to the list
            images.append(img)

    return images


Downloading Tracker Configuration Files: bytetrack.yaml and botsort.yaml from Ultralytics Repository

In [ ]:
!wget https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/trackers/bytetrack.yaml
!wget https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/trackers/botsort.yaml

YOLOv8 Object Tracking and Visualization: Using bytetrack and botsort Trackers on Test Images

In [ ]:
torch.cuda.empty_cache()
gc.collect()

This code performs object tracking using the YOLOv8 model for each image in a specified dataset folder. It utilizes two different trackers ('bytetrack' and 'botsort') provided as YAML configuration files. The annotations obtained from the tracking are stored in lists and then used to create two separate annotated video outputs using OpenCV's VideoWriter.

In [ ]:
# Load images from the specified folder
images = load_images_from_folder('./datasets/data/test/images')

# Initialize YOLOv8 model with a specified checkpoint
model = YOLO("runs/detect/train14/weights/best.pt")

# Lists to store annotated frames for each tracker
video_byte = []
video_bot = []

# Loop through each image in the dataset
for image in images:
    # Run YOLOv8 tracking on the frame, persisting tracks between frames
    results_byte = model.track(image, persist=True, save=True, save_crop=True, project="runs/detect", name="inference", exist_ok=True, tracker='bytetrack.yaml')
    results_bot = model.track(image, persist=True, save=True, save_crop=True, project="runs/detect", name="inference", exist_ok=True, tracker='botsort.yaml')

    # Visualize the results on the frame
    annotated_frame_byte = results_byte[0].plot()
    annotated_frame_bot = results_bot[0].plot()

    # Append annotated frames to the respective lists
    video_byte.append(annotated_frame_byte)
    video_bot.append(annotated_frame_bot)

    # Display the annotated frame
    cv2.imshow("YOLOv8 Tracking using bytetrack tracker", annotated_frame_byte)
    cv2.imshow("YOLOv8 Tracking using botsort tracker", annotated_frame_bot)

# Create video writers for the output videos
out_byte = cv2.VideoWriter('project_byte.avi', cv2.VideoWriter_fourcc(*'DIVX'), 8, (256, 256))
out_bot = cv2.VideoWriter('project_bot.avi', cv2.VideoWriter_fourcc(*'DIVX'), 8, (256, 256))

# Write annotated frames to the output videos
for i in range(len(video_byte)):
    out_byte.write(video_byte[i])
    out_bot.write(video_bot[i])

# Release the video writers
out_byte.release()
out_bot.release()


We're using a library to figure out two important things: how accurate the tracking is (MOTA) and how precise it is (MOTP). The tricky part was getting our hands on the right data format. We had to make sure the labels that tell us where the objects are in the real world and where our model thinks they are match up in a way the library understands. It took some careful preparation to get the data in the right shape. Despite the challenges, our goal is to give a clear picture of how well the tracking model does in real-life situations, showing how close it gets to the right answer and how exact its guesses are.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

# Training: record performance
starting at train 72 - 89

This code performs hyperparameter tuning and training of YOLOv8 models with various combinations of hyperparameters. It utilizes nested loops to iterate through different combinations of models, epochs, batch sizes, learning rates, and optimizers.

In [ ]:
models = ["yolov8x.pt", "yolov8n.pt"]
y_epochs = [10,20,30]
y_batch = [2,4,6]
y_learn = [0.001,0.01,0.1]
y_opt = ["Adam","SGD","RMSProp"]

for y_model in models:

    for epoch in y_epochs:

        for batch in y_batch:

            for lr0 in y_learn:

                for opt in y_opt:

                    model = YOLO(y_model)
                    model.train(data = "data.yaml", epochs = epoch, batch = batch, lr0 = lr0, optimizer = opt)
                    print("--------------------------------------------------------------------------------------------------------------------------------")
                    print("model: {}, epochs: {}, batch: {}, learning rate: {}, optimizer: {}".format(y_model, epoch, batch, lr0, opt))
                    print("--------------------------------------------------------------------------------------------------------------------------------")

New https://pypi.org/project/ultralytics/8.0.218 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208 🚀 Python-3.8.18 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 7972MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=data.yaml, epochs=10, patience=50, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train72, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False

--------------------------------------------------------------------------------------------------------------------------------
model: yolov8x.pt, epochs: 10, batch: 2, learning rate: 0.001, optimizer: Adam
--------------------------------------------------------------------------------------------------------------------------------


New https://pypi.org/project/ultralytics/8.0.218 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208 🚀 Python-3.8.18 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 7972MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=data.yaml, epochs=10, patience=50, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train73, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False,

--------------------------------------------------------------------------------------------------------------------------------
model: yolov8x.pt, epochs: 10, batch: 2, learning rate: 0.001, optimizer: SGD
--------------------------------------------------------------------------------------------------------------------------------


New https://pypi.org/project/ultralytics/8.0.218 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208 🚀 Python-3.8.18 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 7972MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=data.yaml, epochs=10, patience=50, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train74, exist_ok=False, pretrained=True, optimizer=RMSProp, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=Fa

...

--------------------------------------------------------------------------------------------------------------------------------
model: yolov8x.pt, epochs: 10, batch: 4, learning rate: 0.01, optimizer: Adam
--------------------------------------------------------------------------------------------------------------------------------


New https://pypi.org/project/ultralytics/8.0.218 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208 🚀 Python-3.8.18 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 7972MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=data.yaml, epochs=10, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train88, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False,

--------------------------------------------------------------------------------------------------------------------------------
model: yolov8x.pt, epochs: 10, batch: 4, learning rate: 0.01, optimizer: SGD
--------------------------------------------------------------------------------------------------------------------------------


New https://pypi.org/project/ultralytics/8.0.218 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208 🚀 Python-3.8.18 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 7972MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=data.yaml, epochs=10, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train89, exist_ok=False, pretrained=True, optimizer=RMSProp, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=Fa

RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 703295296 vs 703295192

This code is performing multiple object tracking evaluation using the YOLOv8 model's tracking capabilities and the MOTMetrics library.

In [ ]:
import cv2
import os
model = YOLO("runs/detect/train14/weights/best.pt")
i = 0
for image in sorted(os.listdir('./datasets/data/test/images')):
    img = cv2.imread(os.path.join('./datasets/data/test/images', image))
    if i > 17:
        results_byte = model.track(img, persist=True, save=True, save_crop=True, project="runs/detect", name="inference", exist_ok=True, tracker='bytetrack.yaml', save_txt = True)
    i += 1
    if i > 23:
        i = 0
    model = YOLO("runs/detect/train14/weights/best.pt")
    

In [ ]:
pred = []

f = open(os.path.join('./runs/detect/inference/labels/image0.txt'),'r')

frame = 0

pred_mot = []

for line in f:

    line = line.strip()
    line = line.split()
    
    pred_mot.append([frame, int(line[5]), float(line[1]), float(line[2]), float(line[3]), float(line[4])])
    
    frame += 1

[[0, 1, 0.482274, 0.465779, 0.214265, 0.207781], [1, 1, 0.490777, 0.456626, 0.227856, 0.19846], [2, 1, 0.498438, 0.444312, 0.201918, 0.179492], [3, 1, 0.497055, 0.446929, 0.216776, 0.179446], [4, 1, 0.482835, 0.451588, 0.215653, 0.173287], [5, 1, 0.467578, 0.451564, 0.183371, 0.161832], [6, 1, 0.431422, 0.438638, 0.301905, 0.199226], [7, 1, 0.443282, 0.443592, 0.31274, 0.227706], [8, 1, 0.423571, 0.443033, 0.351003, 0.245764], [9, 1, 0.406313, 0.445539, 0.359482, 0.241453], [10, 1, 0.393336, 0.442515, 0.32518, 0.221246], [11, 1, 0.405306, 0.426554, 0.29502, 0.222233], [12, 1, 0.877736, 0.337628, 0.243937, 0.330563], [13, 2, 0.172073, 0.424649, 0.34175, 0.10597], [14, 1, 0.877629, 0.337439, 0.244244, 0.330098], [15, 2, 0.173433, 0.425714, 0.346866, 0.127861], [16, 1, 0.877304, 0.33769, 0.245094, 0.332088], [17, 2, 0.168021, 0.42447, 0.336041, 0.137977], [18, 1, 0.876195, 0.33796, 0.247325, 0.331272], [19, 2, 0.155407, 0.424547, 0.310326, 0.141997], [20, 1, 0.874943, 0.337553, 0.250114, 

In [ ]:
import motmetrics as mm
import numpy as np

acc = mm.MOTAccumulator(auto_id=True)

for pred, gt in zip(pred_mot, gt_mot):

    pred = np.array(pred)
    pred = pred[:, np.newaxis]
    pred = np.ravel(pred)
    
    gt = np.array(gt)
    gt = gt[:, np.newaxis]
    gt = np.ravel(gt)
    
    print(gt[2:])
    
    C = mm.distances.iou_matrix(gt[2:], pred[2:], max_iou=0.5)

    gt = np.array(gt)
    pred = np.array(pred)

    acc.update(gt[:,0].astype('int').tolist(), \
                  pred[:,0].astype('int').tolist(), C)

mh = mm.metrics.create()

summary = mh.compute(acc, metrics=['num_frames', 'idf1', 'idp', 'idr', \
                                     'recall', 'precision', 'num_objects', \
                                     'mostly_tracked', 'partially_tracked', \
                                     'mostly_lost', 'num_false_positives', \
                                     'num_misses', 'num_switches', \
                                     'num_fragmentations', 'mota', 'motp' \
                                    ], \
                      name='acc')

strsummary = mm.io.render_summary(
    summary,
    #formatters={'mota' : '{:.2%}'.format},
    namemap={'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', \
             'precision': 'Prcn', 'num_objects': 'GT', \
             'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
             'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
             'num_misses': 'FN', 'num_switches' : 'IDsw', \
             'num_fragmentations' : 'FM', 'mota': 'MOTA', 'motp' : 'MOTP',  \
            }
)

print(strsummary)